<a href="https://colab.research.google.com/github/dimitramuni/732A92-Text-Mining/blob/main/BERT_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#using data from http://snap.stanford.edu/data/web-FineFoods.html
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Data Cleaning
with open('/content/drive/MyDrive/Finefood/finefood.txt', encoding='latin-1') as source:
    f=source.read()
    raw_list=f.split('\n\n')
    #creating a dictionary with productID and userID as keys 
    #{('productID','userID'):('profileName','helpfuness','score','time','summary','text')}
    
    my_dict={}
    for row in raw_list:
      #disregaring the row which has more than 8 features
      if(len(row.split('\n'))==8):
        my_dict[row.split('\n')[0].split(': ')[1],row.split('\n')[1].split(': ')[1]]\
        =row.split('\n')[2].split(': ')[1],row.split('\n')[3].split(': ')[1],row.\
        split('\n')[4].split(': ')[1],row.split('\n')[5].split(': ')[1],row.split('\n')[6].\
        split(': ')[1],row.split('\n')[7].split(': ')[1]  
      


    

In [ ]:
FF_k=pd.DataFrame(my_dict.keys(),columns=['productID','userID'])
FF_v=pd.DataFrame.from_dict(my_dict,orient='index',columns=['profileName','helpfulness','score','time','summary','text']).reset_index()
FineFoods=pd.merge(FF_k,FF_v,left_index=True, right_index=True).drop(columns='index')
FineFoods['score']=FineFoods['score'].astype('float')
FineFoods[['summary','text']]=FineFoods[['summary','text']].astype('string')
FineFoods.head()
FineFoods.dtypes


productID       object
userID          object
profileName     object
helpfulness     object
score          float64
time            object
summary         string
text            string
dtype: object

In [ ]:
FineFoods_Reduced=FineFoods.loc[FineFoods.score!=3]
FineFoods_Reduced['sentiment']=np.select([FineFoods_Reduced.score>3,FineFoods_Reduced.score<3],['positive','negative'])
FineFoods_Reduced['sentiment_binary']=np.select([FineFoods_Reduced.score>3,FineFoods_Reduced.score<3],[1,0])

FineFoods_Reduced.head()
#https://newbedev.com/how-to-use-if-else-in-pandas-dataframe-code-example

<ipython-input-5-6c3de1eb83d3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FineFoods_Reduced['sentiment']=np.select([FineFoods_Reduced.score>3,FineFoods_Reduced.score<3],['positive','negative'])
<ipython-input-5-6c3de1eb83d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FineFoods_Reduced['sentiment_binary']=np.select([FineFoods_Reduced.score>3,FineFoods_Reduced.score<3],[1,0])


,productID,userID,profileName,helpfulness,score,time,summary,text,sentiment,sentiment_binary
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,positive,1
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,negative,0
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,positive,1
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,negative,0
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wid...,positive,1


In [ ]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,ytrain,ytest=train_test_split(FineFoods_Reduced['text'],FineFoods_Reduced['sentiment_binary'],test_size=0.3, stratify=FineFoods_Reduced['sentiment'])

# Undersampling Imbalanced Dataset

In [ ]:
from imblearn.under_sampling import RandomUnderSampler


#FineFoods['score']=FineFoods['score'].astype('string')
Xtemp=np.array(Xtrain).reshape(-1,1)
ytemp=np.array(ytrain).reshape(-1,1)
RuSampler=RandomUnderSampler()
Xtr,ytr=RuSampler.fit_resample(Xtemp,ytemp)
ytr=np.array(ytr).reshape((-1,1))
ytest=np.array(ytest).reshape((-1,1))

# BERT 

In [ ]:
!pip install -q -U "tensorflow==2.9.0"
!pip install -q -U "tensorflow-text==2.9.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 42.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_text as text
#if(text.__version__!=tf.__version__):
#  raise RuntimeError('tensorflow_text and tensorflow version mismatch!')
import tensorflow_hub as hub
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from sklearn.metrics import classification_report

In [ ]:
#Reference: https://www.tensorflow.org/text/tutorials/classify_text_with_bert

def building_classifier_model(bert_type):
  preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
  if(bert_type=='B1'):
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1")
  
  elif(bert_type=='B2'):
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/2")
  
  elif(bert_type=='B3'):
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/2")
 
  elif(bert_type=='B4'):
    encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2")


  text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name='text input')
  pre_processed_text=preprocessor(text_input)
  encoded_text=encoder(pre_processed_text)

  output_dropout_layer=tf.keras.layers.Dropout(0.1, name='Dropout_Layer')(encoded_text['pooled_output'])
  output_dense_layer=tf.keras.layers.Dense(1,activation='sigmoid',name='Dense_Layer')(output_dropout_layer)
  model=tf.keras.Model(inputs=[text_input],outputs=[output_dense_layer])
  return model

In [ ]:
def evaluate_classifier_model(model):
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[tf.keras.metrics.BinaryAccuracy(),
                                                                                     tf.keras.metrics.AUC(),
                                                                     tf.keras.metrics.Precision(),
                                                                     tf.keras.metrics.Recall()])

  model.fit(Xtr,ytr,epochs=5)
  model.evaluate(Xtest,ytest)
  test_prediction=model.predict(Xtest)
  predicted_test=[1 if x>0.5 else 0 for x in test_prediction.flatten()]
  return classification_report(ytest,predicted_test)

In [ ]:
model_B1=building_classifier_model('B1')

In [ ]:
model_B2=building_classifier_model('B2')

In [ ]:
model_B3=building_classifier_model('B3')

In [ ]:
model_B4=building_classifier_model('B4')

In [ ]:
print(evaluate_classifier_model(model_B1))

Epoch 1/5
3550/3550 [==============================] - 492s 138ms/step - loss: 0.6138 - binary_accuracy: 0.6599 - auc_2: 0.7214 - precision_3: 0.6580 - recall_3: 0.6660
Epoch 2/5
3550/3550 [==============================] - 493s 139ms/step - loss: 0.5910 - binary_accuracy: 0.6829 - auc_2: 0.7502 - precision_3: 0.6820 - recall_3: 0.6855
Epoch 3/5
3550/3550 [==============================] - 487s 137ms/step - loss: 0.5879 - binary_accuracy: 0.6858 - auc_2: 0.7540 - precision_3: 0.6850 - recall_3: 0.6879
Epoch 4/5
3550/3550 [==============================] - 497s 140ms/step - loss: 0.5873 - binary_accuracy: 0.6862 - auc_2: 0.7549 - precision_3: 0.6856 - recall_3: 0.6878
Epoch 5/5
4864/4864 [==============================] - 681s 140ms/step
              precision    recall  f1-score   support

           0       0.31      0.70      0.43     24340
           1       0.93      0.71      0.80    131291

    accuracy                           0.71    155631
   macro avg       0.62      0.70  

In [ ]:
print(evaluate_classifier_model(model_B2))

Epoch 1/5
3550/3550 [==============================] - 1334s 375ms/step - loss: 0.5637 - binary_accuracy: 0.7064 - auc_3: 0.7801 - precision_4: 0.7065 - recall_4: 0.7061
Epoch 2/5
3550/3550 [==============================] - 1343s 378ms/step - loss: 0.5338 - binary_accuracy: 0.7328 - auc_3: 0.8084 - precision_4: 0.7344 - recall_4: 0.7293
Epoch 3/5
3550/3550 [==============================] - 1340s 378ms/step - loss: 0.5315 - binary_accuracy: 0.7323 - auc_3: 0.8102 - precision_4: 0.7338 - recall_4: 0.7292
Epoch 4/5
3550/3550 [==============================] - 1338s 377ms/step - loss: 0.5298 - binary_accuracy: 0.7353 - auc_3: 0.8118 - precision_4: 0.7364 - recall_4: 0.7330
Epoch 5/5
4864/4864 [==============================] - 1808s 372ms/step
              precision    recall  f1-score   support

           0       0.36      0.76      0.49     24340
           1       0.94      0.75      0.83    131291

    accuracy                           0.75    155631
   macro avg       0.65      0

In [ ]:
print(evaluate_classifier_model(model_B3))

Epoch 1/5
3550/3550 [==============================] - 1145s 321ms/step - loss: 0.6000 - binary_accuracy: 0.6758 - auc: 0.7399 - precision: 0.6735 - recall: 0.6824
Epoch 2/5
3550/3550 [==============================] - 1130s 318ms/step - loss: 0.5755 - binary_accuracy: 0.6978 - auc: 0.7680 - precision: 0.6979 - recall: 0.6976
Epoch 3/5
3550/3550 [==============================] - 1119s 315ms/step - loss: 0.5716 - binary_accuracy: 0.7020 - auc: 0.7723 - precision: 0.7017 - recall: 0.7028
Epoch 4/5
3550/3550 [==============================] - 1131s 319ms/step - loss: 0.5719 - binary_accuracy: 0.7016 - auc: 0.7721 - precision: 0.7015 - recall: 0.7016
Epoch 5/5
4864/4864 [==============================] - 1552s 319ms/step


In [ ]:
print(evaluate_classifier_model(model_B4))